# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')

In [2]:
import player_data as player
import pandas as pd
from datetime import datetime
import statsmodels.api as sm

# Get data

In [3]:
file_name = 'csvs/element_gameweek_' + datetime.today().strftime('%Y-%m-%d') + '.csv'

In [4]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [5]:
current_event = 20
element_types = [2, 3, 4]
threshold_minutes = 75

In [6]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
    & (element_gameweek_df['element_type'].isin(element_types))
    & (element_gameweek_df['minutes'] >= threshold_minutes)
]

In [7]:
element_gameweek_df = player.add_categorical_variables(element_gameweek_df, 'element_type')
element_gameweek_df = player.add_categorical_variables(element_gameweek_df, 'element')
element_gameweek_df = player.add_categorical_variables(element_gameweek_df, 'opposition_team')

In [8]:
player.add_home_categorical_variable(element_gameweek_df)

# Models

## Simple Poisson model

### Preparing data

In [9]:
spm_df = element_gameweek_df.copy()

In [10]:
spm_redundant_columns = [
    'row_id', 'assists', 'attempted_passes', 'big_chances_created',
    'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
    'clearances_blocks_interceptions', 'completed_passes', 'creativity',
    'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
    'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
    'ict_index', 'id', 'influence', 'key_passes', 'kickoff_time',
    'kickoff_time_formatted', 'loaned_in', 'loaned_out', 'minutes', 'offside',
    'open_play_crosses', 'opponent_team', 'own_goals', 'penalties_conceded',
    'penalties_missed', 'penalties_saved', 'recoveries', 'red_cards', 'round',
    'saves', 'selected', 'tackled', 'tackles', 'target_missed', 'team_a_score',
    'team_h_score', 'threat', 'total_points', 'transfers_balance',
    'transfers_in', 'transfers_out', 'value', 'was_home', 'winning_goals',
    'yellow_cards', 'event', 'team_a', 'team_a_difficulty', 'team_h',
    'team_h_difficulty', 'element_type', 'team', 'web_name', 'safe_web_name',
    'own_team', 'opposition_team'
]

In [11]:
spm_df = player.remove_redundant_columns(spm_df, spm_redundant_columns)

In [12]:
spm_response_df, spm_explanatory_df = player.get_response_explanatory_dfs(spm_df, 'goals_scored')

### Fitting model

In [ ]:
spm_results = sm.GLM(
    spm_response_df,
    spm_explanatory_df,
    family=sm.families.Poisson(),
    missing='drop'
).fit()

/Users/thomaswoodruff/code/footbot/venv/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:426: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/Users/thomaswoodruff/code/footbot/venv/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:131: RuntimeWarning: divide by zero encountered in true_divide
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/Users/thomaswoodruff/code/footbot/venv/lib/python3.7/site-packages/statsmodels/regression/_tools.py:49: RuntimeWarning: invalid value encountered in multiply
  self.wexog = w_half[:, None] * exog


In [ ]:
print(spm_results.summary())